In [2]:
import json

import cv2
import numpy as np
import os
import base64
import asyncio
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic  # holistic: 인체 다양한 부분 감지(얼굴,손 등)


# mp_drawing = mp.solutions.drawing_utils # mp.holistic에서 감지한 부분 시각화

def process_frame(image_data):
    # base64 형식의 이미지 데이터를 bytes로 디코딩
    image_bytes = base64.b64decode(image_data)

    # bytes를 NumPy 배열로 변환
    np_arr = np.frombuffer(image_bytes, np.uint8)

    # NumPy 배열을 이미지로 변환
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

    # 이미지 처리 로직 수행
    # 예시로 그레이스케일 변환을 수행합니다.
    # gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 처리된 이미지를 다시 base64 형식으로 인코딩
    # _, encoded_img = cv2.imencode('.jpg', gray_img)
    # processed_image_data = base64.b64encode(encoded_img).decode('utf-8')

    return img


def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False  # 이미지 수정 불가
    results = model.process(image)  # 모델을 사용해 입력 이미지에 대한 예측 수행
    image.flags.writeable = True  # 이미지 다시 수정가능
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR COVERSION RGB 2 BGR
    return image, results

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in
                     results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    lh = np.array([[res.x, res.y, res.z] for res in
                   results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in
                   results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(
        21 * 3)
    return np.concatenate([pose, lh, rh])

In [26]:
current_directory = os.getcwd()
print("현재 작업 디렉토리:", current_directory)
DATA_PATH = os.path.join('Model\data')
save_path = os.path.join('Model\key_data')

현재 작업 디렉토리: f:\문서\중앙대\4-1학기\캡스톤\temp


In [27]:

video_file_names = os.listdir(DATA_PATH)
for video_file_name in video_file_names:
    video_files = os.listdir(os.path.join(DATA_PATH, video_file_name))
    for video_file in video_files:
        sequence = []  # collect 30 frames 얘로 prediction
        sentence = []  #

        name = os.path.join(DATA_PATH, video_file_name, video_file)
        print(name)
        cap = cv2.VideoCapture(name)
        frame_count = 0
        # 추출할 프레임 간격 설정
        frame_interval = 3  # 5프레임당 1장 추출
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while(cap.isOpened()):
                ret, frame = cap.read()
                if not ret:
                    break
                frame_count += 1
                if frame_count % frame_interval == 0:
                    image, results = mediapipe_detection(frame, holistic)
                    keypoints = extract_keypoints(results)
                    sequence.append(keypoints)
        sequence = sequence[-30:]  # 마지막 30 frame으로 prediction 한다
        cap.release()
        save_directory = os.path.join(save_path, video_file_name)
        print(f"sp:{save_path} // vf:{video_file_name}")
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)
        np_path = os.path.join(save_directory, f'{video_file.split(".")[0]}.npy')
        np.save(np_path, sequence)


Model\data\나\나_1.mp4
sp:Model\key_data // vf:나
Model\data\나\나_10.mp4
sp:Model\key_data // vf:나
Model\data\나\나_11.mp4
sp:Model\key_data // vf:나
Model\data\나\나_12.mp4
sp:Model\key_data // vf:나
Model\data\나\나_13.mp4
sp:Model\key_data // vf:나
Model\data\나\나_14.mp4
sp:Model\key_data // vf:나
Model\data\나\나_15.mp4
sp:Model\key_data // vf:나
Model\data\나\나_16.mp4
sp:Model\key_data // vf:나
Model\data\나\나_17.mp4
sp:Model\key_data // vf:나
Model\data\나\나_18.mp4
sp:Model\key_data // vf:나
Model\data\나\나_19.mp4
sp:Model\key_data // vf:나
Model\data\나\나_2.mp4
sp:Model\key_data // vf:나
Model\data\나\나_20.mp4
sp:Model\key_data // vf:나
Model\data\나\나_21.mp4
sp:Model\key_data // vf:나
Model\data\나\나_22.mp4
sp:Model\key_data // vf:나
Model\data\나\나_23.mp4
sp:Model\key_data // vf:나
Model\data\나\나_24.mp4
sp:Model\key_data // vf:나
Model\data\나\나_25.mp4
sp:Model\key_data // vf:나
Model\data\나\나_26.mp4
sp:Model\key_data // vf:나
Model\data\나\나_27.mp4
sp:Model\key_data // vf:나
Model\data\나\나_28.mp4
sp:Model\key_data //